## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jan. 5th Jan. 12th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,January-10-2025,Condo/Co-op,3900 County Line Rd Unit 5d,Tequesta,FL,33469.0,300000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/3900-County-...,Beaches MLS,RX-11017526,N,Y,26.969589,-80.092142


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [9]:
df2 = df

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-01-10    28
2025-01-06    22
2025-01-08    20
2025-01-09    16
2025-01-07    12
2025-01-13     1
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
3,PAST SALE,2025-01-10,Condo/Co-op,150 Bradley Pl Unit 905/906,Palm Beach,FL,33480.0,14000000.0,5.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Beaches MLS,RX-11043708,N,Y,26.720485,-80.041578
94,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Beaches MLS,RX-10971987,N,Y,26.344627,-80.077096
55,PAST SALE,2025-01-09,Condo/Co-op,201 S Narcissus Ave #1102,West Palm Beach,FL,33401.0,3364412.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-11026542,N,Y,26.711982,-80.051026
45,PAST SALE,2025-01-07,Condo/Co-op,44 Cocoanut Row Unit 219b,Palm Beach,FL,33480.0,2400000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Beaches MLS,RX-11028940,N,Y,26.715571,-80.042932
56,PAST SALE,2025-01-07,Condo/Co-op,3800 Washington Rd #1212,West Palm Beach,FL,33405.0,1740000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/3800...,Beaches MLS,RX-11027061,N,Y,26.677294,-80.049335
34,PAST SALE,2025-01-10,Condo/Co-op,1225 S Ocean Blvd #205,Delray Beach,FL,33483.0,1635000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11034074,N,Y,26.443001,-80.061098
11,PAST SALE,2025-01-08,Condo/Co-op,300 SE 5th Ave #1160,Boca Raton,FL,33432.0,1482500.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11038852,N,Y,26.346087,-80.076579
54,PAST SALE,2025-01-10,Condo/Co-op,4301 N Ocean Blvd #1108,Boca Raton,FL,33431.0,1400000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Beaches MLS,RX-11029884,N,Y,26.389260,-80.067914
99,PAST SALE,2025-01-08,Condo/Co-op,100 Lakeshore Dr #1251,North Palm Beach,FL,33408.0,1400000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Palm-Beach/100...,Beaches MLS,RX-10945285,N,Y,26.830324,-80.053868
39,PAST SALE,2025-01-10,Condo/Co-op,220 Macfarlane Dr Unit S-606,Delray Beach,FL,33483.0,1325000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/220-Mac...,Beaches MLS,RX-11037709,N,Y,26.457735,-80.063109


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-01-10,Condo/Co-op,150 Bradley Pl Unit 905/906,Palm Beach,FL,33480.0,14000000.0,5.0,5.5,...,-80.041578,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Christopher Leavitt,Douglas Elliman,None,None,Catherine Applegate,Douglas Elliman,None,None
1,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,...,-80.077096,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None
2,PAST SALE,2025-01-09,Condo/Co-op,201 S Narcissus Ave #1102,West Palm Beach,FL,33401.0,3364412.0,2.0,2.5,...,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None
3,PAST SALE,2025-01-07,Condo/Co-op,44 Cocoanut Row Unit 219b,Palm Beach,FL,33480.0,2400000.0,2.0,2.0,...,-80.042932,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Linda Olsson,"Linda R. Olsson, Inc.",None,None,Dana Landry,Dana E. Landry,None,None
4,PAST SALE,2025-01-07,Condo/Co-op,3800 Washington Rd #1212,West Palm Beach,FL,33405.0,1740000.0,3.0,2.0,...,-80.049335,https://www.redfin.com/FL/West-Palm-Beach/3800...,Alexis Waller,Brown Harris Stevens of PB,Lucy Bauer,Brown Harris Stevens of PB,Alexis Waller,Brown Harris Stevens of PB,None,None
5,PAST SALE,2025-01-10,Condo/Co-op,1225 S Ocean Blvd #205,Delray Beach,FL,33483.0,1635000.0,2.0,2.5,...,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,T. Jed Weaver,The Keyes Company,None,None,Vincenza Antonacci,Compass Florida LLC,Brian Pearl,Compass Florida LLC
6,PAST SALE,2025-01-08,Condo/Co-op,300 SE 5th Ave #1160,Boca Raton,FL,33432.0,1482500.0,3.0,3.5,...,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Albert Fidalic,Coldwell Banker Realty /Delray Beach,None,None
7,PAST SALE,2025-01-10,Condo/Co-op,4301 N Ocean Blvd #1108,Boca Raton,FL,33431.0,1400000.0,3.0,3.0,...,-80.067914,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Nancy Ghen,Coldwell Banker/BR,None,None,Nancy Ghen,Coldwell Banker/BR,None,None
8,PAST SALE,2025-01-08,Condo/Co-op,100 Lakeshore Dr #1251,North Palm Beach,FL,33408.0,1400000.0,3.0,3.5,...,-80.053868,https://www.redfin.com/FL/North-Palm-Beach/100...,Lee Feely,"Illustrated Properties, LLC",None,None,Lee Feely,"Illustrated Properties, LLC",None,None
9,PAST SALE,2025-01-10,Condo/Co-op,220 Macfarlane Dr Unit S-606,Delray Beach,FL,33483.0,1325000.0,2.0,2.0,...,-80.063109,https://www.redfin.com/FL/Delray-Beach/220-Mac...,Scott Kleinman,Illustrated Properties,None,None,Rania Ikonomidis,"United Realty Group, Inc",None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

99


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$619,637


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$348


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$61,344,062


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [24]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach
Input Previous Week Condo Sales Total (number units sold):
95
Input Previous Week Condo Average Sales Price:
641448
Input Previous Week Condo Average PSF:
390
Input Previous Week Condo Sales total (ex: 198_000_000)
61_000_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [25]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [26]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [27]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [28]:
pd.set_option('display.max_columns',None)

In [29]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-10,Condo/Co-op,150 Bradley Pl Unit 905/906,Palm Beach,FL,33480.0,14000000.0,5.0,5.5,Palm Beach Biltmore Condo,4287.0,NaN,1926.0,NaN,3266.0,6954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Beaches MLS,RX-11043708,N,Y,26.720485,-80.041578,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Christopher Leavitt,Douglas Elliman,None,None,Catherine Applegate,Douglas Elliman,None,None,1,orange


In [30]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [31]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-10,Condo/Co-op,150 Bradley Pl Unit 905/906,Palm Beach,FL,33480.0,14000000.0,5.0,5.5,Palm Beach Biltmore Condo,4287.0,NaN,1926.0,NaN,3266.0,6954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Beaches MLS,RX-11043708,N,Y,26.720485,-80.041578,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Christopher Leavitt,Douglas Elliman,None,None,Catherine Applegate,Douglas Elliman,None,None,1,orange
1,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,Mizner Grand Condo,5186.0,NaN,1999.0,NaN,848.0,6178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Beaches MLS,RX-10971987,N,Y,26.344627,-80.077096,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,2,blue
2,PAST SALE,2025-01-09,Condo/Co-op,201 S Narcissus Ave #1102,West Palm Beach,FL,33401.0,3364412.0,2.0,2.5,Esplanade Grande Condo,2211.0,NaN,2004.0,NaN,1522.0,2511.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-11026542,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,3,blue
3,PAST SALE,2025-01-07,Condo/Co-op,44 Cocoanut Row Unit 219b,Palm Beach,FL,33480.0,2400000.0,2.0,2.0,Palm Beach Towers Condo,NaN,NaN,1956.0,NaN,NaN,2190.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Beaches MLS,RX-11028940,N,Y,26.715571,-80.042932,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Linda Olsson,"Linda R. Olsson, Inc.",None,None,Dana Landry,Dana E. Landry,None,None,4,blue
4,PAST SALE,2025-01-07,Condo/Co-op,3800 Washington Rd #1212,West Palm Beach,FL,33405.0,1740000.0,3.0,2.0,Portofino South Condo Estates OF South Palm Bea,1594.0,NaN,1971.0,NaN,1092.0,1833.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/3800...,Beaches MLS,RX-11027061,N,Y,26.677294,-80.049335,https://www.redfin.com/FL/West-Palm-Beach/3800...,Alexis Waller,Brown Harris Stevens of PB,Lucy Bauer,Brown Harris Stevens of PB,Alexis Waller,Brown Harris Stevens of PB,None,None,5,blue
5,PAST SALE,2025-01-10,Condo/Co-op,1225 S Ocean Blvd #205,Delray Beach,FL,33483.0,1635000.0,2.0,2.5,Banyan House Condo,1911.0,117000.0,1967.0,NaN,856.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11034074,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,T. Jed Weaver,The Keyes Company,None,None,Vincenza Antonacci,Compass Florida LLC,Brian Pearl,Compass Florida LLC,6,blue
6,PAST SALE,2025-01-08,Condo/Co-op,300 SE 5th Ave #1160,Boca Raton,FL,33432.0,1482500.0,3.0,3.5,Mizner Tower,2465.0,NaN,1989.0,NaN,601.0,3486.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11038852,N,Y,26.346087,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Albert Fidalic,Coldwell Banker Realty /Delray Beach,None,None,7,blue
7,PAST SALE,2025-01-10,Condo/Co-op,4301 N Ocean Blvd #1108,Boca Raton,FL,33431.0,1400000.0,3.0,3.0,Sea Ranch Club OF Boca,2660.0,NaN,1981.0,NaN,526.0,2394.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Beaches MLS,RX-11029884,N,Y,26.389260,-80.067914,https://www.redfin.com/FL/Boca-Raton/4301-N-Oc...,Nancy Ghen,Coldwell Banker/BR,None,None,Nancy Ghen,Coldwell Banker/BR,None,None,8,blue
8,PAST SALE,2025-01-08,Condo/Co-op,100 Lakeshore Dr #12

In [32]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [33]:
m.save('index.html')

## Data snagger

In [34]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [35]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-10,Condo/Co-op,150 Bradley Pl Unit 905/906,Palm Beach,FL,33480.0,14000000.0,5.0,5.5,Palm Beach Biltmore Condo,4287.0,NaN,1926.0,NaN,3266.0,6954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Beaches MLS,RX-11043708,N,Y,26.720485,-80.041578,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Christopher Leavitt,Douglas Elliman,None,None,Catherine Applegate,Douglas Elliman,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [36]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_01132024


# CREATE TEMPLATE 

In [37]:
muni_set = set(df_top_ten['CITY'])

In [38]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [39]:
df_top_ten.reset_index(inplace=True,drop=True)

In [40]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [41]:
top_sale

'Palm Beach Biltmore Condo at 150 Bradley Pl Unit 905/906 in Palm Beach'

In [42]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [43]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [44]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-10,Condo/Co-op,150 Bradley Pl Unit 905/906,Palm Beach,FL,33480.0,14000000.0,5.0,5.5,Palm Beach Biltmore Condo,4287.0,NaN,1926.0,NaN,3266.0,6954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Beaches MLS,RX-11043708,N,Y,26.720485,-80.041578,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Christopher Leavitt,Douglas Elliman,None,None,Catherine Applegate,Douglas Elliman,None,None,1,orange,Palm Beach Biltmore Condo at 150 Bradley Pl Un...
1,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,Mizner Grand Condo,5186.0,NaN,1999.0,NaN,848.0,6178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Beaches MLS,RX-10971987,N,Y,26.344627,-80.077096,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,2,blue,Mizner Grand Condo at 600 SE 5th Ave Unit 108s...
2,PAST SALE,2025-01-09,Condo/Co-op,201 S Narcissus Ave #1102,West Palm Beach,FL,33401.0,3364412.0,2.0,2.5,Esplanade Grande Condo,2211.0,NaN,2004.0,NaN,1522.0,2511.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-11026542,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,3,blue,Esplanade Grande Condo at 201 S Narcissus Ave ...
3,PAST SALE,2025-01-07,Condo/Co-op,44 Cocoanut Row Unit 219b,Palm Beach,FL,33480.0,2400000.0,2.0,2.0,Palm Beach Towers Condo,NaN,NaN,1956.0,NaN,NaN,2190.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Beaches MLS,RX-11028940,N,Y,26.715571,-80.042932,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Linda Olsson,"Linda R. Olsson, Inc.",None,None,Dana Landry,Dana E. Landry,None,None,4,blue,Palm Beach Towers Condo at 44 Cocoanut Row Uni...
4,PAST SALE,2025-01-07,Condo/Co-op,3800 Washington Rd #1212,West Palm Beach,FL,33405.0,1740000.0,3.0,2.0,Portofino South Condo Estates OF South Palm Bea,1594.0,NaN,1971.0,NaN,1092.0,1833.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/3800...,Beaches MLS,RX-11027061,N,Y,26.677294,-80.049335,https://www.redfin.com/FL/West-Palm-Beach/3800...,Alexis Waller,Brown Harris Stevens of PB,Lucy Bauer,Brown Harris Stevens of PB,Alexis Waller,Brown Harris Stevens of PB,None,None,5,blue,Portofino South Condo Estates OF South Palm Be...
5,PAST SALE,2025-01-10,Condo/Co-op,1225 S Ocean Blvd #205,Delray Beach,FL,33483.0,1635000.0,2.0,2.5,Banyan House Condo,1911.0,117000.0,1967.0,NaN,856.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11034074,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,T. Jed Weaver,The Keyes Company,None,None,Vincenza Antonacci,Compass Florida LLC,Brian Pearl,Compass Florida LLC,6,blue,Banyan House Condo at 1225 S Ocean Blvd #205 i...
6,PAST SALE,2025-01-08,Condo/Co-op,300 SE 5th Ave #1160,Boca Raton,FL,33432.0,1482500.0,3.0,3.5,Mizner Tower,2465.0,NaN,1989.0,NaN,601.0,3486.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11038852,N,Y,26.346087,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Albert Fidalic,Coldwell Banker Realty /Delray Beach,None,None,7,blue,Mizner Tower at 300 SE 5th Ave #1160 in Boca R...
7,PAST SALE,2025-01-10,Condo/Co-op,4301 N Ocean Blvd #1108,Boca Raton,FL,33431.0,1400000.0,3.0,3.0,Se

In [45]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Palm Beach Biltmore Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,325,000 to $14,000,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Delray Beach, Boca Raton, Palm Beach, North Palm Beach, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 99 condo sales totaling $61,344,062 million from Jan. 5th to Jan. 12th. The previous week, brokers closed 95 condo sales totaling $61,000,000.

Last week’s units sold for an average of $619,637, lower than the $641,448 average price from th

In [46]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [47]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [48]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [49]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-10,Condo/Co-op,150 Bradley Pl Unit 905/906,Palm Beach,FL,33480.0,14000000.0,5.0,5.5,Palm Beach Biltmore Condo,4287.0,NaN,1926.0,NaN,3266.0,6954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Beaches MLS,RX-11043708,N,Y,26.720485,-80.041578,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Christopher Leavitt,Douglas Elliman,None,None,Catherine Applegate,Douglas Elliman,None,None,1,orange,Palm Beach Biltmore Condo at 150 Bradley Pl Un...
1,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,Mizner Grand Condo,5186.0,NaN,1999.0,NaN,848.0,6178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Beaches MLS,RX-10971987,N,Y,26.344627,-80.077096,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,2,blue,Mizner Grand Condo at 600 SE 5th Ave Unit 108s...
2,PAST SALE,2025-01-09,Condo/Co-op,201 S Narcissus Ave #1102,West Palm Beach,FL,33401.0,3364412.0,2.0,2.5,Esplanade Grande Condo,2211.0,NaN,2004.0,NaN,1522.0,2511.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-11026542,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,3,blue,Esplanade Grande Condo at 201 S Narcissus Ave ...
3,PAST SALE,2025-01-07,Condo/Co-op,44 Cocoanut Row Unit 219b,Palm Beach,FL,33480.0,2400000.0,2.0,2.0,Palm Beach Towers Condo,NaN,NaN,1956.0,NaN,NaN,2190.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Beaches MLS,RX-11028940,N,Y,26.715571,-80.042932,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Linda Olsson,"Linda R. Olsson, Inc.",None,None,Dana Landry,Dana E. Landry,None,None,4,blue,Palm Beach Towers Condo at 44 Cocoanut Row Uni...
4,PAST SALE,2025-01-07,Condo/Co-op,3800 Washington Rd #1212,West Palm Beach,FL,33405.0,1740000.0,3.0,2.0,Portofino South Condo Estates OF South Palm Bea,1594.0,NaN,1971.0,NaN,1092.0,1833.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/3800...,Beaches MLS,RX-11027061,N,Y,26.677294,-80.049335,https://www.redfin.com/FL/West-Palm-Beach/3800...,Alexis Waller,Brown Harris Stevens of PB,Lucy Bauer,Brown Harris Stevens of PB,Alexis Waller,Brown Harris Stevens of PB,None,None,5,blue,Portofino South Condo Estates OF South Palm Be...
5,PAST SALE,2025-01-10,Condo/Co-op,1225 S Ocean Blvd #205,Delray Beach,FL,33483.0,1635000.0,2.0,2.5,Banyan House Condo,1911.0,117000.0,1967.0,NaN,856.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11034074,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,T. Jed Weaver,The Keyes Company,None,None,Vincenza Antonacci,Compass Florida LLC,Brian Pearl,Compass Florida LLC,6,blue,Banyan House Condo at 1225 S Ocean Blvd #205 i...
6,PAST SALE,2025-01-08,Condo/Co-op,300 SE 5th Ave #1160,Boca Raton,FL,33432.0,1482500.0,3.0,3.5,Mizner Tower,2465.0,NaN,1989.0,NaN,601.0,3486.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11038852,N,Y,26.346087,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Albert Fidalic,Coldwell Banker Realty /Delray Beach,None,None,7,blue,Mizner Tower at 300 SE 5th Ave #1160 in Boca R...
7,PAST SALE,2025-01-10,Condo/Co-op,4301 N Ocean Blvd #1108,Boca Raton,FL,33431.0,1400000.0,3.0,3.0,Se

In [50]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Palm Beach Biltmore Condo closing tops Palm Beach weekly condo sales 
DEK: Top 10 sales ranged from $1,325,000 to $14,000,000
FEATURED HED:
SEO HED: Palm Beach County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach, weekly condo sales, Delray Beach, Boca Raton, Palm Beach, North Palm Beach, West Palm Beach

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County’s NEWS PEG HERE.

Brokers closed 99 condo sales totaling $61,344,062 million from Jan. 5th to Jan. 12th. The previous week, brokers closed 95 condo sales totaling $61,000,000.

Last week’s units sold for an average of $619,637, lower than the $641,448 average price from th

In [58]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Palm-Beach/150-Bradley-Pl-33480/unit-905/home/42432865


In [66]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Boca-Raton/600-SE-5th-Ave-33432/unit-108S/home/42538474


In [60]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Delray-Beach/220-Macfarlane-Dr-33483/unit-S606/home/42366825


In [61]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/North-Palm-Beach/100-Lakeshore-Dr-33408/unit-1251/home/42479276


In [62]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Palm-Beach/150-Bradley-Pl-33480/unit-905/home/42432865


In [63]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/North-Palm-Beach/100-Lakeshore-Dr-33408/unit-1251/home/42479276


In [64]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-10,Condo/Co-op,150 Bradley Pl Unit 905/906,Palm Beach,FL,33480.0,14000000.0,5.0,5.5,Palm Beach Biltmore Condo,4287.0,NaN,1926.0,NaN,3266.0,6954.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Beaches MLS,RX-11043708,N,Y,26.720485,-80.041578,https://www.redfin.com/FL/Palm-Beach/150-Bradl...,Christopher Leavitt,Douglas Elliman,None,None,Catherine Applegate,Douglas Elliman,None,None,1,orange,Palm Beach Biltmore Condo at 150 Bradley Pl Un...
1,PAST SALE,2025-01-06,Condo/Co-op,600 SE 5th Ave Unit 108s,Boca Raton,FL,33432.0,4400000.0,3.0,3.5,Mizner Grand Condo,5186.0,NaN,1999.0,NaN,848.0,6178.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Beaches MLS,RX-10971987,N,Y,26.344627,-80.077096,https://www.redfin.com/FL/Boca-Raton/600-SE-5t...,Jonathan Postma,Coldwell Banker/BR,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,2,blue,Mizner Grand Condo at 600 SE 5th Ave Unit 108s...
2,PAST SALE,2025-01-09,Condo/Co-op,201 S Narcissus Ave #1102,West Palm Beach,FL,33401.0,3364412.0,2.0,2.5,Esplanade Grande Condo,2211.0,NaN,2004.0,NaN,1522.0,2511.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/201-...,Beaches MLS,RX-11026542,N,Y,26.711982,-80.051026,https://www.redfin.com/FL/West-Palm-Beach/201-...,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,Steffanie Ngo-Hatchie,United Realty Group Inc,None,None,3,blue,Esplanade Grande Condo at 201 S Narcissus Ave ...
3,PAST SALE,2025-01-07,Condo/Co-op,44 Cocoanut Row Unit 219b,Palm Beach,FL,33480.0,2400000.0,2.0,2.0,Palm Beach Towers Condo,NaN,NaN,1956.0,NaN,NaN,2190.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Beaches MLS,RX-11028940,N,Y,26.715571,-80.042932,https://www.redfin.com/FL/Palm-Beach/44-Cocoan...,Linda Olsson,"Linda R. Olsson, Inc.",None,None,Dana Landry,Dana E. Landry,None,None,4,blue,Palm Beach Towers Condo at 44 Cocoanut Row Uni...
4,PAST SALE,2025-01-07,Condo/Co-op,3800 Washington Rd #1212,West Palm Beach,FL,33405.0,1740000.0,3.0,2.0,Portofino South Condo Estates OF South Palm Bea,1594.0,NaN,1971.0,NaN,1092.0,1833.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/3800...,Beaches MLS,RX-11027061,N,Y,26.677294,-80.049335,https://www.redfin.com/FL/West-Palm-Beach/3800...,Alexis Waller,Brown Harris Stevens of PB,Lucy Bauer,Brown Harris Stevens of PB,Alexis Waller,Brown Harris Stevens of PB,None,None,5,blue,Portofino South Condo Estates OF South Palm Be...
5,PAST SALE,2025-01-10,Condo/Co-op,1225 S Ocean Blvd #205,Delray Beach,FL,33483.0,1635000.0,2.0,2.5,Banyan House Condo,1911.0,117000.0,1967.0,NaN,856.0,1783.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/1225-S-...,Beaches MLS,RX-11034074,N,Y,26.443001,-80.061098,https://www.redfin.com/FL/Delray-Beach/1225-S-...,T. Jed Weaver,The Keyes Company,None,None,Vincenza Antonacci,Compass Florida LLC,Brian Pearl,Compass Florida LLC,6,blue,Banyan House Condo at 1225 S Ocean Blvd #205 i...
6,PAST SALE,2025-01-08,Condo/Co-op,300 SE 5th Ave #1160,Boca Raton,FL,33432.0,1482500.0,3.0,3.5,Mizner Tower,2465.0,NaN,1989.0,NaN,601.0,3486.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Beaches MLS,RX-11038852,N,Y,26.346087,-80.076579,https://www.redfin.com/FL/Boca-Raton/300-SE-5t...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Albert Fidalic,Coldwell Banker Realty /Delray Beach,None,None,7,blue,Mizner Tower at 300 SE 5th Ave #1160 in Boca R...
7,PAST SALE,2025-01-10,Condo/Co-op,4301 N Ocean Blvd #1108,Boca Raton,FL,33431.0,1400000.0,3.0,3.0,Se

## Time on Market Calculator

In [67]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 3, 26) ## List (Earlier) date
date2 = datetime(2025, 1, 6) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

286


## Clean CSV for Datawrapper Chart

In [94]:
chart_df = df_top_ten

In [95]:
chart_df = chart_df.fillna(" ")

In [96]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [97]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [98]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [99]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [100]:
chart_df['$/SQUARE FEET'].astype(float)

ValueError: could not convert string to float: ''

In [93]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

ValueError: Unknown format code 'f' for object of type 'str'

In [101]:
csv_date_string = today.strftime("%m_%d_%Y")

In [102]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [103]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-10,150 Bradley Pl Unit 905/906 in Palm Beach,14000000.0,5.0,5.5,4287.0,1926.0,3266.0,Beaches MLS,RX-11043708,Christopher Leavitt Douglas Elliman,Catherine Applegate Douglas Elliman
1,2025-01-06,600 SE 5th Ave Unit 108s in Boca Raton,4400000.0,3.0,3.5,5186.0,1999.0,848.0,Beaches MLS,RX-10971987,Jonathan Postma Coldwell Banker/BR,Jonathan Postma Coldwell Banker/BR
2,2025-01-09,201 S Narcissus Ave #1102 in West Palm Beach,3364412.0,2.0,2.5,2211.0,2004.0,1522.0,Beaches MLS,RX-11026542,Steffanie Ngo-Hatchie United Realty Group Inc,Steffanie Ngo-Hatchie United Realty Group Inc
3,2025-01-07,44 Cocoanut Row Unit 219b in Palm Beach,2400000.0,2.0,2.0,,1956.0,,Beaches MLS,RX-11028940,"Linda Olsson Linda R. Olsson, Inc.",Dana Landry Dana E. Landry
4,2025-01-07,3800 Washington Rd #1212 in West Palm Beach,1740000.0,3.0,2.0,1594.0,1971.0,1092.0,Beaches MLS,RX-11027061,Alexis Waller Brown Harris Stevens of PB Lucy ...,Alexis Waller Brown Harris Stevens of PB
5,2025-01-10,1225 S Ocean Blvd #205 in Delray Beach,1635000.0,2.0,2.5,1911.0,1967.0,856.0,Beaches MLS,RX-11034074,T. Jed Weaver The Keyes Company,Vincenza Antonacci Compass Florida LLC Brian P...
6,2025-01-08,300 SE 5th Ave #1160 in Boca Raton,1482500.0,3.0,3.5,2465.0,1989.0,601.0,Beaches MLS,RX-11038852,Michael Wells Premier Estate Properties Inc Sc...,Albert Fidalic Coldwell Banker Realty /Delray ...
7,2025-01-10,4301 N Ocean Blvd #1108 in Boca Raton,1400000.0,3.0,3.0,2660.0,1981.0,526.0,Beaches MLS,RX-11029884,Nancy Ghen Coldwell Banker/BR,Nancy Ghen Coldwell Banker/BR
8,2025-01-08,100 Lakeshore Dr #1251 in North Palm Beach,1400000.0,3.0,3.5,3384.0,1982.0,414.0,Beaches MLS,RX-10945285,"Lee Feely Illustrated Properties, LLC","Lee Feely Illustrated Properties, LLC"
9,2025-01-10,220 Macfarlane Dr Unit S-606 in Delray Beach,1325000.0,2.0,2.0,1760.0,1970.0,753.0,Beaches MLS,RX-11037709,Scott Kleinman Illustrated Properties,"Rania Ikonomidis United Realty Group, Inc"
